### Image Noise Reduction
Implement a model that can perform image noise reduction, enhancing the quality of images by reducing noise while preserving important features.

### Objective
Develop and train models that effectively reduce noise in images, improving overall clarity and visual quality.

### Tasks
- Data Cleaning & Transformation: Preprocessing the provided dataset of clean images to generate noisy images for model training.
- Feature Engineering: Analyze noise patterns and apply transformations to optimize the dataset.
- Encoding of images to ensure they are suitable for processing by the models, utilizing libraries such as `OpenCV` for image manipulation.

### Link to the dataset
https://drive.google.com/file/d/1yqLUYqU8_elMhrk1Ebe6p-RdHgmixuta/view?usp=sharing

# Image Noise Reduction Model

This notebook implements various models for reducing noise in images while preserving important features.

## Table of Contents
1. Introduction
2. Setup
3. Data Preprocessing
4. Model Implementations
    - CNN Model
    - U-Net Model
    - Denoising Autoencoder
    - Non-Local Means Denoising
5. Hyperparameter Tuning
6. Training and Evaluation
7. Conclusion


In [ ]:
# 1. Introduction
#please if someone can write a quick intro for this i have made the basics for all the models

# 2. Setup
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

# 3. Data Preprocessing
def load_data(directory):
    images = []
    for folder in os.listdir(directory):
        for file in os.listdir(os.path.join(directory, folder)):
            img = cv2.imread(os.path.join(directory, folder, file))
            if img is not None:
                images.append(img)
    return np.array(images)

# Load training and validation sets
X_train = load_data('') #'path/to/training_set
X_val = load_data('') #path/to/validation_set

def add_noise(img):
    row, col, ch = img.shape
    mean = 0
    var = 0.1
    sigma = var ** 0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    noisy = img + gauss
    return noisy

# Create noisy dataset
X_train_noisy = np.array([add_noise(img) for img in X_train])
X_val_noisy = np.array([add_noise(img) for img in X_val])

# 4. Model Implementations
## A. CNN Model
def cnn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same'))
    model.compile(optimizer='adam', loss='mse')
    return model

# Instantiate and train the CNN model
cnn_input_shape = (128, 128, 3)  # Adjust as necessary
cnn = cnn_model(cnn_input_shape)
cnn.fit(X_train_noisy, X_train, epochs=20, batch_size=32, validation_data=(X_val_noisy, X_val))

## B. U-Net Model
def unet_model(input_shape):
    inputs = layers.Input(input_shape)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    u1 = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(p1)
    u1 = layers.concatenate([u1, c1])
    outputs = layers.Conv2D(3, (1, 1), activation='sigmoid')(u1)
    model = models.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='mse')
    return model

# Instantiate and train the U-Net model
unet = unet_model(cnn_input_shape)
unet.fit(X_train_noisy, X_train, epochs=20, batch_size=32, validation_data=(X_val_noisy, X_val))

## C. Denoising Autoencoder
def autoencoder(input_shape):
    input_img = layers.Input(shape=input_shape)
    encoded = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    encoded = layers.MaxPooling2D((2, 2))(encoded)
    decoded = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    decoded = layers.UpSampling2D((2, 2))(decoded)
    decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(decoded)
    autoencoder = models.Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

# Instantiate and train the Autoencoder model
autoencoder_model = autoencoder(cnn_input_shape)
autoencoder_model.fit(X_train_noisy, X_train, epochs=20, batch_size=32, validation_data=(X_val_noisy, X_val))

## D. Non-Local Means Denoising
def non_local_means(img):
    return cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)

# Example usage
sample_image = X_val_noisy[0]
denoised_image = non_local_means(sample_image)

# Show original and denoised images
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('Noisy Image')
plt.imshow(sample_image.astype(np.uint8))
plt.subplot(1, 2, 2)
plt.title('Denoised Image')
plt.imshow(denoised_image)
plt.show()

# 5. Hyperparameter Tuning
# implement GridSearchCV for hyperparameter tuning as shown in the previous guidance.

# 6. Training and Evaluation
# Evaluate model performance on validation set.

# 7. Conclusion
# Summarize the findings and the performance of each model.
